# Day 8 - recursive data structure

Structures containing structures containing ... recursion.

So, I naturally use a stack! Or, a `deque`, which is faster than the builtin list at adding and removing individual entries when used as a stack.

The stack stores the child and metadata counts as a `[children, meta]` mutable list. You first have to process the children before you can process the metadata for a node, so we need to keep this structure updated until the time to handle the metadata is here. 

Then each time we enter the loop, we check the `children` counter

* If the counter is zero, pop the item from the stack and we can sum `meta` entries from the data stream to the checksum
* otherwise, decrement the counter and add the next two values from the stream to the stack.

In [1]:
from collections import deque
from itertools import islice

def checksum(datastream):
    it = iter(datastream)
    stack = deque([[next(it), next(it)]])
    summed = 0
    while stack:
        if not stack[-1][0]:
            # children done, add the metadata
            _, meta = stack.pop()
            summed += sum(islice(it, None, meta))
        else:
            # decrease counter and process next node
            stack[-1][0] -= 1
            stack.append([next(it), next(it)])
    return summed

In [2]:
testdatastream = [int(d) for d in '''\
2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2'''.split()]
assert checksum(testdatastream) == 138

In [3]:
import aocd

data = aocd.get_data(day=8, year=2018)
datastream = [int(d) for d in data.split()]

In [4]:
print('Part 1:', checksum(datastream))

Part 1: 45750


## More stacking about

Now we need to add one more piece of information to our stack entries: a list of child node values, so we can calculat the value for the current completed node. We don't need to keep the whole tree here!

The steps then become:

* Check the `children` counter
* If it is zero, pop the item from the stack. Collect the metadata numbers from the stream, and calculate our value by doing one of two things:
    * If the list of child node values is not empty, sum the metadata numbers.
    * If the list is not empty, use the metadata numbers as 1-based indices into the list to sum.

In [5]:
def calc_license(datastream):
    it = iter(datastream)
    stack = deque([[next(it), next(it), []]])
    while True:
        if not stack[-1][0]:
            # children done, calculate value
            _, meta, values = stack.pop()
            if not values:
                value = sum(islice(it, None, meta))
            else:
                # 1-based indexing
                value = sum(
                    values[m - 1] for m in islice(it, None, meta)
                    if 0 < m <= len(values)
                )
            if not stack:
                # last node!
                return value
            stack[-1][-1].append(value)
        else:
            # decrease counter and process next node
            stack[-1][0] -= 1
            stack.append([next(it), next(it), []])

In [6]:
assert calc_license(testdatastream) == 66

In [7]:
print('Part 2:', calc_license(datastream))

Part 2: 23266
